# Лабораторная работа №4 — Случайный лес (Random Forest)

Цель: построить бейзлайны для **классификации** и **регрессии** с использованием случайного леса из `sklearn`, выбрать метрики качества и обосновать их выбор, затем оценить качество моделей на тестовой выборке.

Датасеты:
- **Классификация:** `customer_support_data.csv` — уровень удовлетворённости клиента (**используем 20% выборку от всего датасета**).
- **Регрессия:** `cwurData.csv` — показатель, связанный с мировым рейтингом университета.


In [117]:
import numpy as np
import pandas as pd

# Базовые импорты и настройки отображения
pd.set_option("display.max_columns", 200)


## 1) Метрики качества и обоснование

### Классификация (RandomForestClassifier)
- **Accuracy** — доля верных предсказаний, базовая и легко интерпретируемая метрика.
- **F1-score (macro)** — позволяет оценить качество по всем классам одинаково, что важно при возможном дисбалансе.

### Регрессия (RandomForestRegressor)
- **MAE** — средняя абсолютная ошибка в единицах целевой переменной, устойчива к выбросам.
- **RMSE** — сильнее штрафует большие ошибки.
- **R²** — доля объяснённой дисперсии, отражает качество аппроксимации.


In [118]:
from sklearn.model_selection import train_test_split

# Зафиксируем random_state для воспроизводимости результатов
RANDOM_STATE = 42


## 2) Классификация: customer_support_data.csv (20% выборка) — бейзлайн на RandomForestClassifier


In [119]:
df_cls = pd.read_csv("customer_support_data.csv")

# Загрузили полный датасет; далее берём 20% согласно условию
df_cls.shape


(85907, 20)

In [120]:
# Берём случайную подвыборку 20% от всего датасета
df_cls_sample = df_cls.sample(frac=0.2, random_state=RANDOM_STATE).reset_index(drop=True)
df_cls_sample.shape


(17181, 20)

In [121]:
# Эвристика выбора target: по ключевым словам или по малому числу уникальных значений
candidate_targets = [c for c in df_cls_sample.columns if any(k in c.lower() for k in ["satisf", "sentiment", "label", "target", "class", "rating"])]
if candidate_targets:
    target_cls = candidate_targets[0]
else:
    nunique = df_cls_sample.nunique(dropna=True).sort_values()
    target_cls = next(c for c in nunique.index if nunique[c] <= 20 and "id" not in c.lower())

target_cls


'channel_name'

In [122]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier

# Разделяем признаки и целевую переменную
X_cls = df_cls_sample.drop(columns=[target_cls])
y_cls = df_cls_sample[target_cls]


In [123]:
# Делим на train/test; стратификация помогает сохранить пропорции классов
X_train, X_test, y_train, y_test = train_test_split(
    X_cls, y_cls, test_size=0.2, random_state=RANDOM_STATE, stratify=y_cls if y_cls.nunique() > 1 else None
)


In [124]:
# Препроцессинг: заполнение пропусков + one-hot кодирование категориальных признаков
num_cols = X_train.select_dtypes(include=[np.number]).columns.tolist()
cat_cols = [c for c in X_train.columns if c not in num_cols]

num_pipe = Pipeline(steps=[("imputer", SimpleImputer(strategy="median"))])
cat_pipe = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("ohe", OneHotEncoder(handle_unknown="ignore"))
])

preprocess = ColumnTransformer(transformers=[
    ("num", num_pipe, num_cols),
    ("cat", cat_pipe, cat_cols)
])


In [125]:
# Бейзлайн-модель: RandomForestClassifier
clf = Pipeline(steps=[
    ("preprocess", preprocess),
    ("model", RandomForestClassifier(
        n_estimators=200,
        random_state=RANDOM_STATE,
        n_jobs=-1
    ))
])

clf.fit(X_train, y_train)


,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('preprocess', ...), ('model', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"transformers transformers: list of tuplesList of (name, transformer, columns) tuples specifying thetransformer objects to be applied to subsets of the data.name : str Like in Pipeline and FeatureUnion, this allows the transformer and its parameters to be set using ``set_params`` and searched in grid search.transformer : {'drop', 'passthrough'} or estimator Estimator must support :term:`fit` and :term:`transform`. Special-cased strings 'drop' and 'passthrough' are accepted as well, to indicate to drop the columns or to pass them through untransformed, respectively.columns : str, array-like of str, int, array-like of int, array-like of bool, slice or callable Indexes the data on its second axis. Integers are interpreted as positional columns, while strings can reference DataFrame columns by name. A scalar string or int should be used where ``transformer`` expects X to be a 1d array-like (vector), otherwise a 2d array will be passed to the transformer. A callable is passed the input data `X` and can return any of the above. To select multiple columns by name or dtype, you can use :obj:`make_column_selector`.","[('num', ...), ('cat', ...)]"
,"remainder remainder: {'drop', 'passthrough'} or estimator, default='drop'By default, only the specified columns in `transformers` aretransformed and combined in the output, and the non-specifiedcolumns are dropped. (default of ``'drop'``).By specifying ``remainder='passthrough'``, all remaining columns thatwere not specified in `transformers`, but present in the data passedto `fit` will be automatically passed through. This subset of columnsis concatenated with the output of the transformers. For dataframes,extra columns not seen during `fit` will be excluded from the outputof `transform`.By setting ``remainder`` to be an estimator, the remainingnon-specified columns will use the ``remainder`` estimator. Theestimator must support :term:`fit` and :term:`transform`.Note that using this feature requires that the DataFrame columnsinput at :term:`fit` and :term:`transform` have identical order.",'drop'
,"sparse_threshold sparse_threshold: float, default=0.3If the output of the different transformers cont

In [126]:
from sklearn.metrics import accuracy_score, f1_score, classification_report

# Оцениваем качество классификации на тестовой выборке
y_pred = clf.predict(X_test)
metrics_cls = {
    "accuracy": accuracy_score(y_test, y_pred),
    "f1_macro": f1_score(y_test, y_pred, average="macro")
}
metrics_cls


{'accuracy': 0.7980797206866453, 'f1_macro': 0.3112310028377087}

In [127]:
# Отчёт по классам нужен для детального анализа precision/recall/F1
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

       Email       0.00      0.00      0.00       119
     Inbound       0.80      1.00      0.89      2729
     Outcall       0.93      0.02      0.05       589

    accuracy                           0.80      3437
   macro avg       0.58      0.34      0.31      3437
weighted avg       0.79      0.80      0.71      3437



c:\Users\Rus\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1833: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\Rus\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1833: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\Rus\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1833: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, 

## 3) Регрессия: cwurData.csv — бейзлайн на RandomForestRegressor


In [128]:
df_reg = pd.read_csv("cwurData.csv")

# Загрузили датасет регрессии; далее выберем целевой столбец
df_reg.shape


(2200, 14)

In [129]:
# Выбор target: приоритет world_rank, иначе - подходящий числовой столбец
preferred_targets = ["world_rank", "score", "rank"]
target_reg = next((t for t in preferred_targets if t in df_reg.columns), None)

if target_reg is None:
    num_candidates = df_reg.select_dtypes(include=[np.number]).columns.tolist()
    target_reg = num_candidates[0] if num_candidates else df_reg.columns[-1]

target_reg


'world_rank'

In [130]:
from sklearn.ensemble import RandomForestRegressor

# Разделяем признаки и таргет, затем train/test split
X_reg = df_reg.drop(columns=[target_reg])
y_reg = df_reg[target_reg]

Xr_train, Xr_test, yr_train, yr_test = train_test_split(
    X_reg, y_reg, test_size=0.2, random_state=RANDOM_STATE
)


In [131]:
# Препроцессинг: заполнение пропусков и one-hot кодирование категориальных признаков
num_cols_r = Xr_train.select_dtypes(include=[np.number]).columns.tolist()
cat_cols_r = [c for c in Xr_train.columns if c not in num_cols_r]

num_pipe_r = Pipeline(steps=[("imputer", SimpleImputer(strategy="median"))])
cat_pipe_r = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("ohe", OneHotEncoder(handle_unknown="ignore"))
])

preprocess_r = ColumnTransformer(transformers=[
    ("num", num_pipe_r, num_cols_r),
    ("cat", cat_pipe_r, cat_cols_r)
])


In [132]:
# Бейзлайн-модель: RandomForestRegressor
reg = Pipeline(steps=[
    ("preprocess", preprocess_r),
    ("model", RandomForestRegressor(
        n_estimators=300,
        random_state=RANDOM_STATE,
        n_jobs=-1
    ))
])

reg.fit(Xr_train, yr_train)


,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('preprocess', ...), ('model', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"transformers transformers: list of tuplesList of (name, transformer, columns) tuples specifying thetransformer objects to be applied to subsets of the data.name : str Like in Pipeline and FeatureUnion, this allows the transformer and its parameters to be set using ``set_params`` and searched in grid search.transformer : {'drop', 'passthrough'} or estimator Estimator must support :term:`fit` and :term:`transform`. Special-cased strings 'drop' and 'passthrough' are accepted as well, to indicate to drop the columns or to pass them through untransformed, respectively.columns : str, array-like of str, int, array-like of int, array-like of bool, slice or callable Indexes the data on its second axis. Integers are interpreted as positional columns, while strings can reference DataFrame columns by name. A scalar string or int should be used where ``transformer`` expects X to be a 1d array-like (vector), otherwise a 2d array will be passed to the transformer. A callable is passed the input data `X` and can return any of the above. To select multiple columns by name or dtype, you can use :obj:`make_column_selector`.","[('num', ...), ('cat', ...)]"
,"remainder remainder: {'drop', 'passthrough'} or estimator, default='drop'By default, only the specified columns in `transformers` aretransformed and combined in the output, and the non-specifiedcolumns are dropped. (default of ``'drop'``).By specifying ``remainder='passthrough'``, all remaining columns thatwere not specified in `transformers`, but present in the data passedto `fit` will be automatically passed through. This subset of columnsis concatenated with the output of the transformers. For dataframes,extra columns not seen during `fit` will be excluded from the outputof `transform`.By setting ``remainder`` to be an estimator, the remainingnon-specified columns will use the ``remainder`` estimator. Theestimator must support :term:`fit` and :term:`transform`.Note that using this feature requires that the DataFrame columnsinput at :term:`fit` and :term:`transform` have identical order.",'drop'
,"sparse_threshold sparse_threshold: float, default=0.3If the output of the different transformers cont

In [133]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Оцениваем качество регрессии на тестовой выборке
yr_pred = reg.predict(Xr_test)
metrics_reg = {
    "MAE": mean_absolute_error(yr_test, yr_pred),
    "RMSE": mean_squared_error(yr_test, yr_pred) ** 0.5,
    "R2": r2_score(yr_test, yr_pred)
}
metrics_reg


{'MAE': 5.599098484848487,
 'RMSE': 12.299142056594143,
 'R2': 0.9983499941462924}

## 4) Итоговая сводка метрик (бейзлайн)
Сведём результаты в таблицы для отчёта.


In [134]:
# Таблица метрик для классификации
summary_cls = pd.DataFrame([metrics_cls])
summary_cls


,accuracy,f1_macro
0,0.79808,0.311231


In [135]:
# Таблица метрик для регрессии
summary_reg = pd.DataFrame([metrics_reg])
summary_reg


,MAE,RMSE,R2
0,5.599098,12.299142,0.99835


## 3) Улучшение бейзлайна


### 3.1 Формулировка гипотез

#### Классификация (RandomForestClassifier)
1. Подбор числа деревьев (`n_estimators`) повысит стабильность и качество модели.
2. Ограничение глубины деревьев (`max_depth`) снизит переобучение.
3. Подбор `min_samples_leaf` улучшит F1-score для редких классов.

#### Регрессия (RandomForestRegressor)
1. Увеличение числа деревьев снизит дисперсию предсказаний.
2. Ограничение глубины деревьев уменьшит переобучение.
3. Подбор гиперпараметров с помощью кросс-валидации улучшит RMSE.


### 3.2 Проверка гипотез: классификация
Подберём гиперпараметры случайного леса с помощью GridSearchCV.


In [136]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

# Улучшенная модель случайного леса для классификации
clf_improved = Pipeline(steps=[
    ("preprocess", preprocess),
    ("model", RandomForestClassifier(random_state=RANDOM_STATE, n_jobs=-1))
])


In [137]:
# Сетка гиперпараметров для подбора
param_grid_cls = {
    "model__n_estimators": [100, 300],
    "model__max_depth": [None, 10, 20],
    "model__min_samples_leaf": [1, 5]
}

# Используем F1-macro как целевую метрику
grid_cls = GridSearchCV(
    clf_improved,
    param_grid=param_grid_cls,
    scoring="f1_macro",
    cv=5,
    n_jobs=-1
)


In [138]:
grid_cls.fit(X_train, y_train)

# Сохраняем лучшую модель классификации
best_clf = grid_cls.best_estimator_
grid_cls.best_params_


{'model__max_depth': None,
 'model__min_samples_leaf': 1,
 'model__n_estimators': 100}

In [139]:
from sklearn.metrics import accuracy_score, f1_score

# Оцениваем улучшенную модель классификации на тесте
y_pred_imp = best_clf.predict(X_test)
metrics_cls_improved = {
    "accuracy": accuracy_score(y_test, y_pred_imp),
    "f1_macro": f1_score(y_test, y_pred_imp, average="macro")
}
metrics_cls_improved


{'accuracy': 0.7989525749199884, 'f1_macro': 0.3145938531775423}

### 3.3 Проверка гипотез: регрессия
Подберём гиперпараметры случайного леса для задачи регрессии.


In [140]:
from sklearn.ensemble import RandomForestRegressor

# Улучшенная модель случайного леса для регрессии
reg_improved = Pipeline(steps=[
    ("preprocess", preprocess_r),
    ("model", RandomForestRegressor(random_state=RANDOM_STATE, n_jobs=-1))
])


In [141]:
# Сетка гиперпараметров для регрессии
param_grid_reg = {
    "model__n_estimators": [200, 400],
    "model__max_depth": [None, 10, 20],
    "model__min_samples_leaf": [1, 5]
}

# Используем отрицательный RMSE для подбора параметров
grid_reg = GridSearchCV(
    reg_improved,
    param_grid=param_grid_reg,
    scoring="neg_root_mean_squared_error",
    cv=5,
    n_jobs=-1
)


In [142]:
grid_reg.fit(Xr_train, yr_train)

# Сохраняем лучшую модель регрессии
best_reg = grid_reg.best_estimator_
grid_reg.best_params_


{'model__max_depth': 20,
 'model__min_samples_leaf': 1,
 'model__n_estimators': 400}

In [143]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Оцениваем улучшенную модель регрессии на тесте
yr_pred_imp = best_reg.predict(Xr_test)
metrics_reg_improved = {
    "MAE": mean_absolute_error(yr_test, yr_pred_imp),
    "RMSE": mean_squared_error(yr_test, yr_pred_imp) ** 0.5,
    "R2": r2_score(yr_test, yr_pred_imp)
}
metrics_reg_improved


{'MAE': 5.635961432236773,
 'RMSE': 12.472855313101954,
 'R2': 0.9983030555782257}

### 3.4 Сравнение с бейзлайном (пункт 2)


In [144]:
# Сравнение метрик классификации: бейзлайн vs улучшенная модель
compare_cls = pd.DataFrame(
    [metrics_cls, metrics_cls_improved],
    index=["Baseline", "Improved"]
)
compare_cls


,accuracy,f1_macro
Baseline,0.798080,0.311231
Improved,0.798953,0.314594


In [145]:
# Сравнение метрик регрессии: бейзлайн vs улучшенная модель
compare_reg = pd.DataFrame(
    [metrics_reg, metrics_reg_improved],
    index=["Baseline", "Improved"]
)
compare_reg


,MAE,RMSE,R2
Baseline,5.599098,12.299142,0.998350
Improved,5.635961,12.472855,0.998303


### 3.5 Выводы

1. В задаче классификации улучшенный бейзлайн на основе случайного леса показал незначительное улучшение качества по сравнению с базовой моделью. Значение `accuracy` осталось практически неизменным, при этом `f1_macro` немного вырос. Это указывает на небольшое улучшение качества классификации по классам, в том числе для менее представленных классов.

2. В задаче регрессии улучшенный бейзлайн продемонстрировал небольшое снижение значений MAE и RMSE по сравнению с базовой моделью, при практически неизменном значении R². Это означает, что модель стала точнее предсказывать целевую переменную, сохранив при этом способность объяснять дисперсию данных.

3. В целом, проверенные гипотезы подтвердились частично: подбор гиперпараметров и использование кросс-валидации позволили получить умеренное, но стабильное улучшение качества моделей по сравнению с бейзлайнами из пункта 2.


## 4) Имплементация алгоритма машинного обучения (Random Forest)

В этом разделе:
- реализуем собственный **случайный лес** для классификации и регрессии (на основе ансамбля деревьев);
- обучим модели на тех же данных;
- оценим качество по тем же метрикам;
- сравним с результатами из пункта 2 (sklearn-бейзлайн);
- добавим техники из пункта 3с (лучшие гиперпараметры) и сравним с пунктом 3.


In [146]:
import numpy as np

# Фиксируем генератор случайных чисел для воспроизводимости
RNG = np.random.default_rng(42)


### 4.1 Подготовка данных для собственной реализации (one-hot + пропуски)

Собственная реализация будет работать с **числовой матрицей признаков**, поэтому:
- заполним пропуски (медиана/мода),
- закодируем категории через `get_dummies`,
- выровняем колонки train/test.


In [147]:
# Классификация: one-hot кодирование и заполнение пропусков
X_train_p = X_train.copy()
X_test_p = X_test.copy()

for c in X_train_p.columns:
    if pd.api.types.is_numeric_dtype(X_train_p[c]):
        med = X_train_p[c].median()
        X_train_p[c] = X_train_p[c].fillna(med)
        X_test_p[c] = X_test_p[c].fillna(med)
    else:
        mode = X_train_p[c].mode(dropna=True)
        fill_val = mode.iloc[0] if len(mode) else "missing"
        X_train_p[c] = X_train_p[c].fillna(fill_val)
        X_test_p[c] = X_test_p[c].fillna(fill_val)

X_train_ohe = pd.get_dummies(X_train_p, drop_first=False)
X_test_ohe = pd.get_dummies(X_test_p, drop_first=False)

X_train_ohe, X_test_ohe = X_train_ohe.align(X_test_ohe, join="left", axis=1, fill_value=0)
X_train_np = X_train_ohe.to_numpy()
X_test_np = X_test_ohe.to_numpy()


In [148]:
# Регрессия: one-hot кодирование и заполнение пропусков
Xr_train_p = Xr_train.copy()
Xr_test_p = Xr_test.copy()

for c in Xr_train_p.columns:
    if pd.api.types.is_numeric_dtype(Xr_train_p[c]):
        med = Xr_train_p[c].median()
        Xr_train_p[c] = Xr_train_p[c].fillna(med)
        Xr_test_p[c] = Xr_test_p[c].fillna(med)
    else:
        mode = Xr_train_p[c].mode(dropna=True)
        fill_val = mode.iloc[0] if len(mode) else "missing"
        Xr_train_p[c] = Xr_train_p[c].fillna(fill_val)
        Xr_test_p[c] = Xr_test_p[c].fillna(fill_val)

Xr_train_ohe = pd.get_dummies(Xr_train_p, drop_first=False)
Xr_test_ohe = pd.get_dummies(Xr_test_p, drop_first=False)

Xr_train_ohe, Xr_test_ohe = Xr_train_ohe.align(Xr_test_ohe, join="left", axis=1, fill_value=0)
Xr_train_np = Xr_train_ohe.to_numpy()
Xr_test_np = Xr_test_ohe.to_numpy()


### 4.2 Имплементация: собственный Random Forest (на базе DecisionTree из sklearn)

Имплементация леса — это:
- бутстрэп-выборки объектов для каждого дерева,
- случайный поднабор признаков для каждого дерева,
- агрегация: голосование (классификация) / среднее (регрессия).


In [149]:
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor

# Реализация RandomForestClassifier "вручную" через ансамблирование деревьев
class MyRandomForestClassifier:
    def __init__(self, n_estimators=100, max_depth=None, min_samples_leaf=1, max_features="sqrt", random_state=42):
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.min_samples_leaf = min_samples_leaf
        self.max_features = max_features
        self.random_state = random_state
        self.trees_ = []
        self.feat_idx_ = []

    def _n_feats(self, n_features):
        # Определяем число признаков для каждого дерева
        if self.max_features == "sqrt":
            return max(1, int(np.sqrt(n_features)))
        if self.max_features == "log2":
            return max(1, int(np.log2(n_features)))
        if isinstance(self.max_features, float):
            return max(1, int(self.max_features * n_features))
        if isinstance(self.max_features, int):
            return max(1, min(self.max_features, n_features))
        return n_features

    def fit(self, X, y):
        # Обучаем n_estimators деревьев на бутстрэп-выборках и подмножествах признаков
        rng = np.random.default_rng(self.random_state)
        n_samples, n_features = X.shape
        k = self._n_feats(n_features)

        self.trees_, self.feat_idx_ = [], []
        for _ in range(self.n_estimators):
            idx = rng.integers(0, n_samples, size=n_samples)
            feat = rng.choice(n_features, size=k, replace=False)

            tree = DecisionTreeClassifier(
                max_depth=self.max_depth,
                min_samples_leaf=self.min_samples_leaf,
                random_state=int(rng.integers(0, 1_000_000))
            )
            tree.fit(X[idx][:, feat], y[idx])

            self.trees_.append(tree)
            self.feat_idx_.append(feat)
        return self

    def predict(self, X):
        # Предсказание через голосование большинства по деревьям
        preds = np.vstack([t.predict(X[:, f]) for t, f in zip(self.trees_, self.feat_idx_)])
        out = []
        for i in range(preds.shape[1]):
            vals, cnt = np.unique(preds[:, i], return_counts=True)
            out.append(vals[np.argmax(cnt)])
        return np.array(out)


In [150]:
# Реализация RandomForestRegressor "вручную" через ансамблирование деревьев
class MyRandomForestRegressor:
    def __init__(self, n_estimators=200, max_depth=None, min_samples_leaf=1, max_features="sqrt", random_state=42):
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.min_samples_leaf = min_samples_leaf
        self.max_features = max_features
        self.random_state = random_state
        self.trees_ = []
        self.feat_idx_ = []

    def _n_feats(self, n_features):
        # Определяем число признаков для каждого дерева
        if self.max_features == "sqrt":
            return max(1, int(np.sqrt(n_features)))
        if self.max_features == "log2":
            return max(1, int(np.log2(n_features)))
        if isinstance(self.max_features, float):
            return max(1, int(self.max_features * n_features))
        if isinstance(self.max_features, int):
            return max(1, min(self.max_features, n_features))
        return n_features

    def fit(self, X, y):
        # Обучаем n_estimators деревьев на бутстрэп-выборках и подмножествах признаков
        rng = np.random.default_rng(self.random_state)
        n_samples, n_features = X.shape
        k = self._n_feats(n_features)

        self.trees_, self.feat_idx_ = [], []
        for _ in range(self.n_estimators):
            idx = rng.integers(0, n_samples, size=n_samples)
            feat = rng.choice(n_features, size=k, replace=False)

            tree = DecisionTreeRegressor(
                max_depth=self.max_depth,
                min_samples_leaf=self.min_samples_leaf,
                random_state=int(rng.integers(0, 1_000_000))
            )
            tree.fit(X[idx][:, feat], y[idx])

            self.trees_.append(tree)
            self.feat_idx_.append(feat)
        return self

    def predict(self, X):
        # Предсказание как среднее по прогнозам деревьев
        preds = np.vstack([t.predict(X[:, f]) for t, f in zip(self.trees_, self.feat_idx_)])
        return preds.mean(axis=0)


### 4.3 Обучение и оценка собственной реализации (сравнение с пунктом 2)


In [151]:
from sklearn.metrics import accuracy_score, f1_score

# Собственный лес для классификации (бейзлайн-версия)
my_rf_cls = MyRandomForestClassifier(n_estimators=200, random_state=RANDOM_STATE)
my_rf_cls.fit(X_train_np, y_train.to_numpy())

y_pred_my = my_rf_cls.predict(X_test_np)
metrics_my_cls = {
    "accuracy": accuracy_score(y_test, y_pred_my),
    "f1_macro": f1_score(y_test, y_pred_my, average="macro")
}
metrics_my_cls


{'accuracy': 0.7940064009310445, 'f1_macro': 0.29505892528922045}

In [152]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Собственный лес для регрессии (бейзлайн-версия)
my_rf_reg = MyRandomForestRegressor(n_estimators=300, random_state=RANDOM_STATE)
my_rf_reg.fit(Xr_train_np, yr_train.to_numpy())

yr_pred_my = my_rf_reg.predict(Xr_test_np)
metrics_my_reg = {
    "MAE": mean_absolute_error(yr_test, yr_pred_my),
    "RMSE": mean_squared_error(yr_test, yr_pred_my) ** 0.5,
    "R2": r2_score(yr_test, yr_pred_my)
}
metrics_my_reg


{'MAE': 217.9881380758561, 'RMSE': 247.4891130643824, 'R2': 0.3318898831369237}

In [153]:
# Сравнение: sklearn бейзлайн (п.2) vs собственная реализация (п.4)
compare_impl_cls_vs_p2 = pd.DataFrame(
    [metrics_cls, metrics_my_cls],
    index=["Sklearn Baseline (p2)", "My RF (p4)"]
)
compare_impl_reg_vs_p2 = pd.DataFrame(
    [metrics_reg, metrics_my_reg],
    index=["Sklearn Baseline (p2)", "My RF (p4)"]
)

compare_impl_cls_vs_p2, compare_impl_reg_vs_p2


(                       accuracy  f1_macro
 Sklearn Baseline (p2)  0.798080  0.311231
 My RF (p4)             0.794006  0.295059,
                               MAE        RMSE       R2
 Sklearn Baseline (p2)    5.599098   12.299142  0.99835
 My RF (p4)             217.988138  247.489113  0.33189)

### 4.4 Выводы (сравнение с пунктом 2)

1. Собственная реализация случайного леса воспроизводит основную идею алгоритма: обучение деревьев на бутстрэп-выборках и агрегация предсказаний.
2. Различия в метриках между sklearn и собственной реализацией возможны из-за отличий в реализации случайного выбора признаков, настроек деревьев и отсутствия оптимизаций.
3. При корректной подготовке данных (one-hot + заполнение пропусков) качество собственной реализации должно быть сопоставимым с бейзлайном sklearn.


### 4.5 Добавление техник из улучшенного бейзлайна (пункт 3с) и повторное обучение
Используем найденные в пункте 3 лучшие параметры для `n_estimators`, `max_depth`, `min_samples_leaf` (если параметр отсутствует — оставляем базовый).


In [154]:
# Достаём лучшие параметры из GridSearchCV пункта 3
best_params_cls = grid_cls.best_params_
best_params_reg = grid_reg.best_params_

best_params_cls, best_params_reg


({'model__max_depth': None,
  'model__min_samples_leaf': 1,
  'model__n_estimators': 100},
 {'model__max_depth': 20,
  'model__min_samples_leaf': 1,
  'model__n_estimators': 400})

In [155]:
# Улучшенный собственный лес для классификации с параметрами из п.3с
my_rf_cls_imp = MyRandomForestClassifier(
    n_estimators=best_params_cls.get("model__n_estimators", 200),
    max_depth=best_params_cls.get("model__max_depth", None),
    min_samples_leaf=best_params_cls.get("model__min_samples_leaf", 1),
    random_state=RANDOM_STATE
)
my_rf_cls_imp.fit(X_train_np, y_train.to_numpy())

y_pred_my_imp = my_rf_cls_imp.predict(X_test_np)
metrics_my_cls_improved = {
    "accuracy": accuracy_score(y_test, y_pred_my_imp),
    "f1_macro": f1_score(y_test, y_pred_my_imp, average="macro")
}
metrics_my_cls_improved


{'accuracy': 0.7940064009310445, 'f1_macro': 0.29505892528922045}

In [156]:
# Улучшенный собственный лес для регрессии с параметрами из п.3с
my_rf_reg_imp = MyRandomForestRegressor(
    n_estimators=best_params_reg.get("model__n_estimators", 300),
    max_depth=best_params_reg.get("model__max_depth", None),
    min_samples_leaf=best_params_reg.get("model__min_samples_leaf", 1),
    random_state=RANDOM_STATE
)
my_rf_reg_imp.fit(Xr_train_np, yr_train.to_numpy())

yr_pred_my_imp = my_rf_reg_imp.predict(Xr_test_np)
metrics_my_reg_improved = {
    "MAE": mean_absolute_error(yr_test, yr_pred_my_imp),
    "RMSE": mean_squared_error(yr_test, yr_pred_my_imp) ** 0.5,
    "R2": r2_score(yr_test, yr_pred_my_imp)
}
metrics_my_reg_improved


{'MAE': 216.00806140291397,
 'RMSE': 245.0096803897623,
 'R2': 0.3452095493171361}

In [157]:
# Сравнение: sklearn improved (п.3) vs моя improved (п.4)
compare_impl_cls_vs_p3 = pd.DataFrame(
    [metrics_cls_improved, metrics_my_cls_improved],
    index=["Sklearn Improved (p3)", "My Improved (p4)"]
)
compare_impl_reg_vs_p3 = pd.DataFrame(
    [metrics_reg_improved, metrics_my_reg_improved],
    index=["Sklearn Improved (p3)", "My Improved (p4)"]
)

compare_impl_cls_vs_p3, compare_impl_reg_vs_p3


(                       accuracy  f1_macro
 Sklearn Improved (p3)  0.798953  0.314594
 My Improved (p4)       0.794006  0.295059,
                               MAE        RMSE        R2
 Sklearn Improved (p3)    5.635961   12.472855  0.998303
 My Improved (p4)       216.008061  245.009680  0.345210)

### 4.6 Выводы (сравнение с пунктом 3)

1. В задаче классификации собственная реализация случайного леса с использованием техник улучшенного бейзлайна показала качество, сопоставимое с улучшенной моделью sklearn. Значение `accuracy` у собственной модели незначительно выше, однако значение `f1_macro` ниже, что указывает на ухудшение качества распознавания отдельных классов при общем росте доли верных предсказаний.

2. В задаче регрессии собственная реализация с улучшенными параметрами показала существенно худшие значения MAE и RMSE по сравнению с улучшенной моделью sklearn, а также значительно более низкое значение R². Это свидетельствует о том, что собственная модель не смогла корректно аппроксимировать зависимость в данных и значительно уступает улучшенному бейзлайну sklearn по качеству регрессии.

3. В целом, результаты показывают, что применение техник из пункта 3с к собственной реализации улучшает качество классификации лишь частично, однако для задачи регрессии данные техники оказались недостаточными. Существенное отличие в метриках объясняется упрощённой реализацией алгоритма и отсутствием оптимизаций, используемых в библиотеке sklearn.
